In [ ]:
%matplotlib inline
import random

import numpy as np

from justice import affine_xform
from justice import summarize
from justice import supernova_data
from justice import visualize

In [ ]:
sn_dataset = supernova_data.SNDataset()
df = sn_dataset.index_df[sn_dataset.index_df['type'] != -9]

type_to_ids = {}
for key, subframe in df.sort_values('type').groupby('type'):
    type_to_ids.setdefault(key, [])
    type_to_ids[key].extend(sorted(frozenset(subframe['id'].tolist())))

In [ ]:
# Choose 2 random light curves of the same type.

# The SN dataset generation may differ, so just hardcode the IDs for now.
#
# rng = random.Random(89794)
# random_type = rng.choice(sorted(type_to_ids.keys()))
# random_ids = rng.sample(type_to_ids[random_type], 2)
random_type = 3
random_ids = [35417, 13810]

lca = supernova_data.format_dense_multi_band_from_lc_dict(
    sn_dataset.lc_dict_for_id(random_ids[0])
)
lcb = supernova_data.format_dense_multi_band_from_lc_dict(
    sn_dataset.lc_dict_for_id(random_ids[1])
)
visualize.plot_single_lc_color_bands(
    lca, "{}, type {}".format(random_ids[0], random_type)
)
visualize.plot_single_lc_color_bands(
    lcb, "{}, type {}".format(random_ids[1], random_type)
)
None  # Here and below: Avoid plot showing up twice by setting cell return value to None.

In [ ]:
# Currently seems to yield disjoint curves.

aff = affine_xform.Aff(
    tx=np.mean(lca.x) - np.mean(lcb.x), ty=np.min(lca.y) - np.min(lcb.y), dx=1, dy=1
)
ivals = aff.as_array()
# aff = summarize.opt_gp(lca, lcb, ivals=ivals, vb=False, options={'maxiter': 10})
aff

In [ ]:
visualize.plot_lcs([lca, affine_xform.transform(lcb, aff)])
None